In [1]:
from dataWrangler import *
from scrape_craigslist import Scrape
from predictions import *
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import math
import glob   
import os
import io
import re

In [2]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
Link='https://sfbay.craigslist.org/eby/cto/d/el-sobrante-2007-lexus-es-350/7308010080.html'

In [6]:
lis=[]
for x in range(3,23,5):
    lis.append(singlePredict(Link,frame,x))

x=0
for l in lis:
    x+=(l[1])
res=int(x/len(lis))
result=((math.floor(res / 100.0)) * 100)    
    
print("-----------------------------------------------")
print(' Recommended:',result,"\n","Actual:",lis[1][-1],"\n",'Low:',min(lis)[1],'-','High:',max(lis)[1])

lexus es Accuracy: 97.71 %.
lexus es Accuracy: 97.5 %.
lexus es Accuracy: 97.79 %.
lexus es Accuracy: 97.77 %.
-----------------------------------------------
 Recommended: 10000 
 Actual: 11500 
 Low: 8626 - High: 10752
